In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
import os
import psutil
import time
import sys
import pathlib
import h5py
%matplotlib inline

from turbo_seti.find_doppler.find_doppler import FindDoppler

In [3]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
# fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=False)
# fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :                1126.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                                1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nfpc :                             1024
            nifs :                                1
     rawdatafile : guppi_59902_24005_HIP50744_0057.0000.raw
     source_name :                         HIP50744
         src_dej :                      21:23:26.88
         src_raj :                     10:21:45.024
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2022-11-

In [ ]:
from pathlib import Path


snr = 10
#nfpc = 65536/64
outdir = '/datax/scratch/jliang/'
count = 0

# threshold of 20 GB in bytes
threshold = 20 * (1024 ** 3)


def has_nfpc(file_path):
    fb = bl.Waterfall(file_path, load_data=False)   # header only
    #print(f"Processing file: {file_path}")
    return "nfpc" in fb.header                   # fb.header is a dict

# for i, file in df['.h5 path'].items():
#     while psutil.virtual_memory().available < threshold:
#         print('Waiting for memory to free up…')
#         time.sleep(60)
#     if has_nfpc(file):
#         print(f"Processing file {i}")
#         fb = bl.Waterfall(file, load_data=False)
#         # nchans = int(fb.header['nchans'])    
#         # nfpc = nchans // 64                     
        

#         # build a base name without its extension
#         base = os.path.splitext(os.path.basename(file))[0]
#         output_path = os.path.join(outdir, f"{base}.{snr}_seticore.dat")

#         # run seticore
#         console = 'seticore ' + file + ' -M 4 -s ' + str(snr) + ' --output ' + outdir + os.path.basename(file)[:-2] + str(snr) + '_seticore.dat'
#         os.system(console)

#         # safely assign into the DataFrame
#         df.loc[i, '.dat path'] = output_path
#         print("Wrote:", output_path)

#         dat_df = pd.read_table(output_path, sep='\s+', names=['Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency','Corrected_Frequency',
#                                                     'Index', 'freq_start', 'freq_end', 'SEFD', 'SEFD_freq', 'Coarse_Channel_Number', 
#                                                     'Full_number_of_hits'], skiprows=9)
#         df['Corrected_Frequency'] = dat_df['Corrected_Frequency']
#         df['Drift_Rate'] = dat_df['Drift_Rate']
#         df['SNR'] = dat_df['SNR']

# # save CSV
# df.to_csv('/datax/scratch/jliang/galaxies_cadences.csv', index=False)


KeyboardInterrupt: 

In [ ]:
cadence_df = pd.read_csv('/datax/scratch/jliang/galaxies_cadences.csv')
for i, file in cadence_df['.h5 path'].items():
    if not has_nfpc(file):
        df.loc[i, '.dat path'] = ''
    for i, file in output_df['.dat path'].items():
    df['.dat path'] = df['.dat path'].apply(lambda p: str(Path(p).parent/'seticore_output'/Path(p).name))
'''
add above code: if no nfpc, change .dat path to nothing
output_df = pd.read_csv('/datax/scratch/jliang/galaxies_cadences.csv')
for i, file in output_df['.dat path'].items():
    df['.dat path'] = df['.dat path'].apply(lambda p: str(Path(p).parent/'seticore_output'/Path(p).name))
'''